In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
from typing import Tuple
from time import time
from numpy.linalg import inv
%config InlineBackend.figure_formats = ['svg']

In [2]:
PQLU = Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]

In [3]:
def randomized_lu(A: np.ndarray, k: int, l: int, seed: int=0) -> PQLU:
    """Performs a randomized rank-k LU decomposition of A.

    Args:
        A: An mXn matrix to decompose.
        k: Rank of the decomposition.
        l: Number of columns to use in the random matrix.
        seed: Random seed.

    Returns:
        A 4-tuple containing P, Q, L, U."""
    rand = np.random.RandomState(seed)
    # 1. Create a matrix G of size n × l whose entries are i.i.d. Gaussian
    # random variables with zero mean and unit standard deviation.
    assert l >= k
    m, n = A.shape
    G = rand.randn(n, l)

    # 2. Y ← AG.    
    Y = A @ G  
    assert Y.shape == (m, l)

    # 3. Apply RRLU decomposition (Theorem 3.1) to Y such that P Y Qy = LyUy.
    #
    # Remark 4.2. In practice, it is sufficient to perform step 3 in Algorithm
    # 4.1 using standard LU decomposition with partial pivoting instead of
    # applying RRLU. The cases where U grows exponentially are extremely rare...
    P, L_y, U_y = la.lu(Y)
    P = P.T
    Q_y = np.identity(l) # TODO: replace with RRLU
    assert P.shape == (m, m)
    assert L_y.shape == (m, l)
    assert U_y.shape == (l, l)
    #assert np.allclose(P @ Y, L_y @ U_y)
    #assert np.allclose(P @ Y @ Q_y, L_y @ U_y)

    # 4. Truncate Ly and Uy by choosing the first k columns and the first k rows,
    # respectively, such that Ly ← Ly(:, 1 : k) and Uy ← Uy(1 : k, :).
    L_y = L_y[:, :k]
    U_y = U_y[:k, :]
    assert L_y.shape == (m, k) 
    assert U_y.shape == (k, l)

    # 5. B ← (L_y †) PA
    L_y_pseudoinverse = la.pinv(L_y)
    assert L_y_pseudoinverse.shape == (k, m)
    B = L_y_pseudoinverse @ P @ A
    assert B.shape == (k, n)

    # 6. Apply LU decomposition to B with column pivoting BQ = L_b U_b.
    Q, U_b, L_b = la.lu(B.T)
    #Q = Q.T
    L_b = L_b.T
    U_b = U_b.T
    assert Q.shape == (n, n)
    assert L_b.shape == (k, k)
    assert U_b.shape == (k, n)
    #assert np.allclose(B @ Q, L_b @ U_b)

    # 7. L ← L_y L_b.
    L = L_y @ L_b
    assert L.shape == (m, k)

    return P, Q, L, U_b

In [4]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

digits = datasets.load_digits()

# flatten the images
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Split data into 50% train and 50% test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False
)

In [5]:
import numpy as np
from numpy.linalg import pinv, norm
from sklearn.base import BaseEstimator, ClassifierMixin

class DictLearner(BaseEstimator, ClassifierMixin):
    
    def dist(self, x: np.ndarray, D: np.ndarray) -> float:
        """Calculates the distance metric between an input
           vector x and a dictionary D

        Args:
            x: The input vector.
            D: A trained dictionary.

        Returns:
            The distance between x and D."""
#         print(D.shape)
#         print(pinv(D).shape)
#         print(x.shape)

        
        return norm((D @ pinv(D))[:x.shape[0],:x.shape[0]] * x - x) ## WTH - dimensions don't align

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Predicts which class the input vector x belongs to
           given a list of dictionaries

        Args:
            X: Matrix of input vectors to classify

        Returns:
            The predicted class of each vector in X."""
        
        
        class_predictions = [self.classes[np.argmin([self.dist(x, self.dictionaries[class_]) for class_ in self.classes])] 
                             for x in X]
        return class_predictions

    def fit(self, X: np.ndarray, y: np.ndarray, k: dict = {}) -> list:
        """Trains a dictionary of size k for each dataset in X

        Args:
            X: A matrix of input vectors.
            y: A vector of class labels.
            k: A map for the size of each class's dictionary. (key,val) = (class, size)

        Returns:
            A DictLearner object with a trained dictionary for each class"""
        
        self.dictionaries = dict()
        
        self.classes = np.unique(y)
        for class_ in self.classes:
            X_class = X[np.where(y == class_)[0]]
            P, Q, L, U = randomized_lu(X_class, k[class_], k[class_] + 5)
#             print(P.T.shape, L.shape)
            self.dictionaries[class_] = P.T @ L
            
        return self

In [6]:
dic = DictLearner()
k = dict.fromkeys(np.unique(y_train), 64) # magic number 64 is size of each dictionary

dic.fit(X_train, y_train, k=k)

DictLearner()

In [7]:
dic.predict(X_train[:10]) # supposed to be 0...9

[3, 1, 5, 1, 0, 6, 1, 5, 1, 6]

In [8]:
dic.score(X_train,y_train)

0.13251670378619154

In [9]:
dic.score(X_test,y_test)

0.1468298109010011